In [7]:
from Module.MySQL_connector import MySQLConnector
import pandas as pd

In [45]:
query = "select * from main_aa \
join genres_onehot_aa on main_aa.steamId = genres_onehot_aa.steamId \
join features_onehot_aa on main_aa.steamId = features_onehot_aa.steamId \
join tags_onehot_aa on main_aa.steamId = tags_onehot_aa.steamId \
join first30days_sales_2A on main_aa.steamId = first30days_sales_2A.steamId"


In [52]:
sql_connector = MySQLConnector('192.168.31.130', 'bigred', 'bigred', 32010)
sql_connector.connectDB('g3_db')
# print(sql_connector.execute_query('show tables;'))
sql_connector.execute_query('show tables;')
main_table = pd.DataFrame(sql_connector.execute_query("select * from main_aa"))
genres_onehot = pd.DataFrame(sql_connector.execute_query("select * from genres_onehot_aa"))
features_onehot = pd.DataFrame(sql_connector.execute_query("select * from features_onehot_aa"))
tags_onehot = pd.DataFrame(sql_connector.execute_query("select * from tags_onehot_aa"))

Connected to g3_db on 192.168.31.130:32010


In [90]:
copiesold = pd.DataFrame(sql_connector.execute_query("select * from first30days_sales_2A"))

In [94]:
copiesold = copiesold.drop(columns=['timeStamp','releaseDate','days_since_release'])


KeyError: "['timeStamp', 'releaseDate', 'days_since_release'] not found in axis"

In [95]:
copiesold

,steamId,sales
0,41050,0.000000e+00
1,41060,5.691300e+04
2,208650,1.796316e+05
3,220200,1.099500e+06
4,223100,5.393900e+04
...,...,...
1179,2881650,1.050977e+06
1180,2903950,4.581000e+03
1181,2933080,9.695800e+04
1182,2969380,2.590000e+03


In [85]:
# 合併時指定 suffixes 參數來處理重複列
merged_table = pd.merge(main_table, genres_onehot, on='steamId', suffixes=('', '_genres'))
merged_table = pd.merge(merged_table, tags_onehot, on='steamId', suffixes=('', '_tags'))
# 再次合併時，處理更多的重複列
merged_table = pd.merge(merged_table, features_onehot, on='steamId', suffixes=('', '_features'))


In [87]:
# 找到所有帶有 '_tags' 或 '_features' 後綴的列名
replicated_columns = [col for col in merged_table.columns if '_tags' in col or '_features' in col]

# 刪除這些列
merged_table.drop(columns=replicated_columns, inplace=True)

In [ ]:
for item in merged_table.columns:
    print(item)

In [97]:
merged_table

,steamId,name,price,reviews,reviewsSteam,followers,avgPlaytime,reviewScore,releaseDate,firstReleaseDate,earlyAccess,copiesSold,revenue,totalRevenue,players,owners,wishlists,Action,Adventure,Casual,Early Access,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,ATV,Abstract,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Auto Battler,Automation,Automobile Sim,BMX,Base Building,Baseball,Based On A Novel,Basketball,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Birds,Blood,Board Game,Boomer Shooter,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Cats,Character Action Game,Character Customization,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Coding,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Cozy,Crafting,Creature Collector,Cricket,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Dice,Difficult,Dinosaurs,Diplomacy,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dwarf,Dynamic Narration,Dystopian,Economy,Education,Electronic,Electronic Music,Elf,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,Extraction Shooter,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football (American),Football (Soccer),Foreign,Fox,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Job Simulator,Kickstarter,LEGO,LGBTQ+,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Mahjong,Management,Mars,Martial Arts,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Musou,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle Platformer,PvE,PvP,Quick-Time Events,RPGMaker,RTS,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Roguelike,Roguelike Deckbuilder,Roguelite,Roguevania,Romance,Rome,Rugby,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Shop Keeper,Short,Side Scroller,Silent Protagonist,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snow,Social Deduction,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fighter,Spelling,Split Screen,Stealth,Steampunk,Story Rich,

In [101]:

full_table = pd.merge(merged_table, copiesold, on="steamId")
full_table

,steamId,name,price,reviews,reviewsSteam,followers,avgPlaytime,reviewScore,releaseDate,firstReleaseDate,earlyAccess,copiesSold,revenue,totalRevenue,players,owners,wishlists,Action,Adventure,Casual,Early Access,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,ATV,Abstract,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Auto Battler,Automation,Automobile Sim,BMX,Base Building,Baseball,Based On A Novel,Basketball,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Birds,Blood,Board Game,Boomer Shooter,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Cats,Character Action Game,Character Customization,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Coding,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Cozy,Crafting,Creature Collector,Cricket,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Dice,Difficult,Dinosaurs,Diplomacy,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dwarf,Dynamic Narration,Dystopian,Economy,Education,Electronic,Electronic Music,Elf,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,Extraction Shooter,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football (American),Football (Soccer),Foreign,Fox,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Job Simulator,Kickstarter,LEGO,LGBTQ+,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Mahjong,Management,Mars,Martial Arts,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Musou,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle Platformer,PvE,PvP,Quick-Time Events,RPGMaker,RTS,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Roguelike,Roguelike Deckbuilder,Roguelite,Roguevania,Romance,Rome,Rugby,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Shop Keeper,Short,Side Scroller,Silent Protagonist,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snow,Social Deduction,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fighter,Spelling,Split Screen,Stealth,Steampunk,Story Rich,

In [105]:
columns_to_drop = ['steamId','name','reviews','reviewsSteam','followers','avgPlaytime','reviewScore','releaseDate','firstReleaseDate','copiesSold','revenue','totalRevenue','players','owners','wishlists']
full_table=full_table.drop(columns=columns_to_drop)

In [113]:
full_table = full_table.dropna()

In [121]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
table_ready_for_ml = full_table["price"] = scaler.fit_transform(full_table[["price"]])
full_table

C:\Users\student\AppData\Local\Temp\ipykernel_21764\2809796661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_ready_for_ml = full_table["price"] = scaler.fit_transform(full_table[["price"]])


,price,earlyAccess,Action,Adventure,Casual,Early Access,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,ATV,Abstract,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Auto Battler,Automation,Automobile Sim,BMX,Base Building,Baseball,Based On A Novel,Basketball,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Birds,Blood,Board Game,Boomer Shooter,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Cats,Character Action Game,Character Customization,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Coding,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Cozy,Crafting,Creature Collector,Cricket,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Dice,Difficult,Dinosaurs,Diplomacy,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dwarf,Dynamic Narration,Dystopian,Economy,Education,Electronic,Electronic Music,Elf,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,Extraction Shooter,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football (American),Football (Soccer),Foreign,Fox,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Job Simulator,Kickstarter,LEGO,LGBTQ+,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Mahjong,Management,Mars,Martial Arts,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Musou,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle Platformer,PvE,PvP,Quick-Time Events,RPGMaker,RTS,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Roguelike,Roguelike Deckbuilder,Roguelite,Roguevania,Romance,Rome,Rugby,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Shop Keeper,Short,Side Scroller,Silent Protagonist,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snow,Social Deduction,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fighter,Spelling,Split Screen,Stealth,Steampunk,Story Rich,Strategy RPG,Stylized,Submarine,Superhero,Supernatural,Surreal,Survival,Survival Horror,Swordplay,Tabletop,Tactical,Tactical RPG,Tanks,Team-Based,Tennis,T

In [133]:
full_table.describe() 
full_table.to_csv('df_ready_for_ml.csv')

In [128]:
X = full_table.drop(columns="sales").values
Y = full_table['sales'].values

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 25)

In [131]:
print(f"Shape of X_train:{X_train.shape}")
print(f"Shape of X_test:{X_test.shape}")
print(f"Shape of y_train:{y_train.shape}")
print(f"Shape of y_test{y_test.shape}")

Shape of X_train:(828, 462)
Shape of X_test:(355, 462)
Shape of y_train:(828,)
Shape of y_test(355,)


In [130]:
# start regression ML
from sklearn.linear_model import LinearRegression
regressor_linear = LinearRegression()
regressor_linear.fit(X_train, y_train)

LinearRegression()

In [132]:
from sklearn.metrics import r2_score

# R2 score，即決定係數，反映Y的全部變異能通過迴歸關係被所有X解釋的比例。
y_pred_linear_train = regressor_linear.predict(X_train)
r2_score_linear_train = r2_score(y_train, y_pred_linear_train)

y_pred_linear_test = regressor_linear.predict(X_test)
r2_score_linear_test = r2_score(y_test, y_pred_linear_test)

# RMSE一般指均方根誤差(標準誤差)，可衡量預測值和實際值之間的平均差異，藉此估計預測模型預測目標值的準確度
# rmse_train = (np.sqrt(mean_squared_error(y_train, y_pred_linear_train)))
# rmse_test = (np.sqrt(mean_squared_error(y_test, y_pred_linear_test)))

print('R2_score (train): ', r2_score_linear_train)
print('R2_score (test): ', r2_score_linear_test)
# print("RMSE: ", rmse_train)
# print("RMSE: ", rmse_test)

R2_score (train):  0.6806990813985216
R2_score (test):  -4.162368719765313e+16
